# Digit Recognizer with CNN

In this notebook, we build and evaluate a Convolutional Neural Network (CNN) to recognize hand-drawn digits from the MNIST-like dataset provided by Kaggle's Digit Recognizer competition. Our objective is to preprocess the training data, augment it, and use a CNN to train a model capable of accurately predicting the digits in the test set. Finally, we format and save the predictions for submission in the required Kaggle format. The steps include data preparation, model training, data augmentation, and evaluation on the validation set before making predictions on the test set.

## First Part - Creating the model

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### **Loading the Training Data**
Here, we load the `train.csv` and `test.csv` files containing the images of digits and their associated labels. The images are stored as pixel values, and the labels represent the drawn digits.


In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

### **Preparing the Data**
The images are resized and normalized to make them compatible with the model (values between 0 and 1). We also perform reshaping to ensure the images have the correct shape (`28x28x1`).


In [3]:
train_images = train_df.iloc[:, 1:].values
train_labels = train_df.iloc[:, 0].values

test_images = test_df.values

train_images = train_images.reshape((-1, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((-1, 28, 28, 1)).astype('float32') / 255

train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)

### **Splitting the Data into Training and Validation Sets**
We split the training data into two subsets: one for training and one for validation. This allows us to check the model's performance during training on data it has not seen.


In [4]:
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42
)

### **Defining the ImageDataGenerator for Data Augmentation**
We use `ImageDataGenerator` to apply transformations to the images during training. These transformations include rotation, translation, zoom, and more, which helps increase the diversity of the data.

In [5]:
datagen = ImageDataGenerator(
    rotation_range=10,  # Rotation de -10 à +10 degrés
    width_shift_range=0.1,  # Déplacement horizontal
    height_shift_range=0.1,  # Déplacement vertical
    zoom_range=0.1,  # Zoom entre 90% et 110%
    shear_range=0.1,  # Décalage (shear)
    horizontal_flip=False,  # Pas de retournement horizontal
    fill_mode='nearest'  # Compléter les pixels manquants par la valeur la plus proche
)

datagen.fit(train_images)

### **Defining the Model**
We define a convolutional neural network (CNN) with two convolutional layers, two pooling layers, a flatten layer, and a final dense layer with 10 neurons to predict digits from 0 to 9.


In [6]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

### **Compiling the Model**
We compile the model using the `Adam` optimizer and the `sparse_categorical_crossentropy` loss function, which is suited for multi-class classification problems.


In [7]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### **Training the Model with Data Augmentation**
We train the model using the data augmentation generator. Each epoch will see new variations of the images thanks to augmentation, helping improve the model's generalization ability.


In [8]:
model.fit(datagen.flow(train_images, train_labels, batch_size=64), epochs=10, validation_data=(val_images, val_labels))

Epoch 1/10
420/420 [==============================] - 11s 24ms/step - loss: 0.6226 - accuracy: 0.7980 - val_loss: 0.1865 - val_accuracy: 0.9421
Epoch 2/10
420/420 [==============================] - 10s 24ms/step - loss: 0.2532 - accuracy: 0.9207 - val_loss: 0.1456 - val_accuracy: 0.9540
Epoch 3/10
420/420 [==============================] - 10s 25ms/step - loss: 0.1836 - accuracy: 0.9430 - val_loss: 0.1037 - val_accuracy: 0.9649
Epoch 4/10
420/420 [==============================] - 10s 24ms/step - loss: 0.1562 - accuracy: 0.9499 - val_loss: 0.1142 - val_accuracy: 0.9640
Epoch 5/10
420/420 [==============================] - 10s 24ms/step - loss: 0.1293 - accuracy: 0.9592 - val_loss: 0.0742 - val_accuracy: 0.9743
Epoch 6/10
420/420 [==============================] - 11s 26ms/step - loss: 0.1253 - accuracy: 0.9611 - val_loss: 0.0733 - val_accuracy: 0.9756
Epoch 7/10
420/420 [==============================] - 11s 26ms/step - loss: 0.1149 - accuracy: 0.9621 - val_loss: 0.0648 - val_accuracy:

## Evaluating the Model
After training, we evaluate the model's performance on the validation set. This helps us understand the accuracy and generalization capabilities of the model.

### Making Predictions on Test Data
With the trained model, we make predictions on the test dataset. Each image is classified into one of the ten digit classes.

In [9]:
predictions = model.predict(test_images)
predicted_classes = np.argmax(predictions, axis=1)

875/875 [==============================] - 3s 3ms/step


In [10]:
submission_df = pd.DataFrame({
    'ImageId': np.arange(1, len(predicted_classes) + 1),
    'Label': predicted_classes
})

### Creating the Submission File
We prepare the submission file by associating each test image with its predicted label. The submission file is formatted as required by Kaggle: each row contains an ImageId and the corresponding predicted Label.


In [11]:
submission_df = pd.DataFrame({
    'ImageId': np.arange(1, len(predicted_classes) + 1),
    'Label': predicted_classes
})

submission_df.to_csv('submission.csv', index=False)

# Conclusion

This code prepares the training data, applies real-time data augmentation during training, trains a convolutional neural network model, and saves the model for future use. Key steps include normalizing the images, splitting the data into training and validation sets, and applying data augmentation during training to improve model performance.
